In [232]:
import pandas as pd
import numpy as np


In [233]:
truck_data = pd.read_csv("truck dataset.csv")
truck_data.head(1)

,order_date,customer_name,customer_phone,customer_primary_contact,delivery_address,Type_description,amount,Truck_type,Date_of_purchase,Employee_last_name,Employee_first_name,Date_of_hire
0,30-06-2019,Smelding Homes,780-743-7548,Bill Johnson,55 Smith Lane,Home foundation and walkways,8,Ford,17-06-1999,Johnson,Emilio,01-02-1985


In [296]:
truck_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   order_date                149 non-null    datetime64[ns]
 1   customer_name             149 non-null    object        
 2   customer_phone            149 non-null    object        
 3   customer_primary_contact  149 non-null    object        
 4   delivery_address          149 non-null    object        
 5   Type_description          149 non-null    object        
 6   amount                    149 non-null    int64         
 7   Truck_type                149 non-null    object        
 8   Date_of_purchase          149 non-null    datetime64[ns]
 9   Employee_last_name        149 non-null    object        
 10  Employee_first_name       149 non-null    object        
 11  Date_of_hire              149 non-null    datetime64[ns]
dtypes: datetime64[ns](3), 

### Change date datatype

In [265]:
truck_data["order_date"]=pd.to_datetime(truck_data["order_date"])
truck_data["Date_of_purchase"]=pd.to_datetime(truck_data["Date_of_purchase"])
truck_data["Date_of_hire"]=pd.to_datetime(truck_data["Date_of_hire"])

### Extract employee data from  main data 

In [266]:
emp = truck_data.groupby(["Employee_last_name","Employee_first_name",
                          "Date_of_hire"])["amount"].count().reset_index()

#### Hear johson hire date is wrong

In [267]:
truck_data.loc[truck_data["Date_of_hire"] == "1985-02-01","Date_of_hire"] = "1985-01-02"

#### Now create emp table emp_id column

In [268]:
emp["emp_id"] = ["E_" + str(i) for i in range(101,101+len(emp))]

In [269]:
emp_table = emp[["emp_id","Employee_first_name","Employee_last_name","Date_of_hire"]]

In [240]:
# emp_table=emp_table.set_index("emp_id")

### Extract customer data then create customer table

In [271]:
cust = truck_data.groupby(["customer_name","customer_phone",
                    "customer_primary_contact"])["amount"].count().reset_index()

#### Hear "Home Builders Superior" contact number wrong and "customer_primary_contact" name is misspell

In [273]:
truck_data.loc[(truck_data["customer_name"]=="Home Builders Superior") & 
               (truck_data["customer_phone"] == "403-543-2987"),
               "customer_phone"] ="587-643-0982"

In [274]:
truck_data.loc[truck_data["customer_primary_contact"] 
               == "Bill Jonson","customer_primary_contact"] = "Bill Johnson"

In [275]:
cust["cus_id"] = ["c" + str(i) for i in range(101,101+len(cust))]

In [276]:
customer_table = cust[["cus_id","customer_name","customer_phone","customer_primary_contact"]]

In [247]:
# customer_table = customer_table.set_index("cus_id")

#### Extract truck data 

In [279]:
truck  = truck_data.groupby(["Truck_type","Type_description"])["amount"].count().reset_index()

In [280]:
truck["t_id"] = ["t" + str(i) for i in range(101,101+len(truck))]

In [281]:
truck_table = truck[["t_id","Truck_type","Type_description"]]

In [253]:
# truck_table = truck_table.set_index("t_id")

#### Now merge all table in main table

#### Merge main table and employee table

In [284]:
t_e = truck_data.merge(emp_table,how="inner",on="Employee_first_name")


#### Now merge last merge table and customer table

In [286]:
t_e_c = t_e.merge(customer_table,how="inner",on="customer_name")

#### Now merge last merge table and truck table

In [288]:
t_e_c_t = t_e_c.merge(truck_table,how = "inner",on=["Truck_type","Type_description"])
t_e_c_t.shape

(149, 19)

#### Now extract main table

In [290]:
t_e_c_t["order_id"] = ["o" + str(i) for i in range(101,101+len(t_e_c_t))]

In [ ]:
order_data = t_e_c_t[["order_id","delivery_address","amount",
                      "Date_of_purchase","emp_id","cus_id","t_id"]]

In [292]:
order_data.head(5)

,order_id,delivery_address,amount,Date_of_purchase,emp_id,cus_id,t_id
0,o101,55 Smith Lane,8,1999-06-17,E_102,c105,t106
1,o102,2122 E. Biscayne,3,2001-12-24,E_103,c103,t106
2,o103,55 Smith Lane,6,2001-12-24,E_101,c105,t108
3,o104,1333 Burr Ridge,4,2002-01-01,E_101,c106,t105
4,o105,1333 Burr Ridge,8,2001-12-24,E_103,c106,t110


In [293]:
emp_table

,emp_id,Employee_first_name,Employee_last_name,Date_of_hire
0,E_101,Antonio,Evarez,1992-03-03
1,E_102,Emilio,Johnson,1985-01-02
2,E_103,John,Robertson,1999-01-06
3,E_104,Allison,Smithson,1997-01-04


In [294]:
customer_table

,cus_id,customer_name,customer_phone,customer_primary_contact
0,c101,Home Builders Superior,587-643-0982,Marcus Connolly
1,c102,Home Makers,250-981-8543,John Yu
2,c103,Mark Akey,403-543-2987,Mark Akey
3,c104,Sheryl Williamson,403-222-8674,Sheryl Williamson
4,c105,Smelding Homes,780-743-7548,Bill Johnson
5,c106,Triple A Homes,780-453-2087,Janielle Smith


In [295]:
truck_table

,t_id,Truck_type,Type_description
0,t101,Chevy,Home foundation and walkways
1,t102,Chevy,Premier marble (concrete with crushed marble a...
2,t103,Chevy,Premier shell (concrete with shell aggregate)
3,t104,Chevy,Premier speckled(concrete with pea-size smooth...
4,t105,Chevy,commercial foundation and infrastructure
5,t106,Ford,Home foundation and walkways
6,t107,Ford,Premier marble (concrete with crushed marble a...
7,t108,Ford,Premier shell (concrete with shell aggregate)
8,t109,Ford,Premier speckled(concrete with pea-size smooth...
9,t110,Ford,commercial foundation and infrastructure
